# 予測関数の実装
MNISTの学習済み重みを用いて、予測(クラス分類を行うこと)関数を実装していく

In [ ]:
try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、DAY1/2_notebookフォルダのsample_weight.pklを選択し、アップロードしてください')
    print('===========')
    files.upload()
except:
    print('ローカル環境での実行です')

Google Colab. 上での実行です
「ファイルを選択」から、DAY1/2_notebookフォルダのsample_weight.pklを選択し、アップロードしてください


Saving sample_weight.pkl to sample_weight.pkl


In [ ]:
%matplotlib inline
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score

# Load the MNIST dataset
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

train = X_train/255
test = X_test/255
train = train.reshape(-1, 28*28)
test = test.reshape(-1, 28*28)
train_labels = lb.fit_transform(y_train)
test_labels = lb.fit_transform(y_test)

11501568/11490434 [==============================] - 0s 0us/step


## 学習済みの重みを読む

In [ ]:
def init_network():
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    return network

network = init_network()
print(network.keys())
print("")

W1, W2, W3 = network["W1"],network["W2"],network["W3"]
b1, b2, b3 = network["b1"],network["b2"],network["b3"] 

print(network["W1"])

dict_keys(['b2', 'W1', 'b1', 'W2', 'W3', 'b3'])

[[-0.00741249 -0.00790439 -0.01307499 ...  0.01978721 -0.04331266
  -0.01350104]
 [-0.01029745 -0.01616653 -0.01228376 ...  0.01920228  0.02809811
   0.01450908]
 [-0.01309184 -0.00244747 -0.0177224  ...  0.00944778  0.01387301
   0.03393568]
 ...
 [ 0.02242565 -0.0296145  -0.06326169 ... -0.01012643  0.01120969
   0.01027199]
 [-0.00761533  0.02028973 -0.01498873 ...  0.02735376 -0.01229855
   0.02407041]
 [ 0.00027915 -0.06848375  0.00911191 ... -0.03183098  0.00743086
  -0.04021148]]


### 行列の形状の確認

In [ ]:
print(W1.shape, W2.shape, W3.shape)
print("            " , b1.shape, "      " ,b2.shape,"    " ,  b3.shape)

(784, 50) (50, 100) (100, 10)
             (50,)        (100,)      (10,)


### [問]
* 入力層のノード数は？
* 1つ目の中間層のノード数は？
* 2つ目の中間層のノード数は？
* 出力層のノード数は？

## 予測関数の実装

### [演習]
* 以下のpredict関数を完成させましょう

In [ ]:
def sigmoid(x):
    return 1 / (1+ np.exp(-x))

def softmax(x):
    c = np.max(x)    
    exp_a = np.exp(x - c)
    return exp_a / np.sum(exp_a)

def predict(network, x):
    """
    クラス分類するための関数
    network : 重み行列を納めたdict
    x : 入力ベクトル
    return : ソフトマックス関数の結果
    """
    W1, W2, W3 = network["W1"],network["W2"],network["W3"]
    b1, b2, b3 = network["b1"],network["b2"],network["b3"]    
    
    # 順伝播の計算
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    return softmax(a3)

### 1データずつ予測を行う

In [ ]:
# テストデータについて、識別を行う
t = np.argmax(test_labels, axis=1)
p = np.array([]).astype(int)
NUM = 1000
for i in range(NUM):
    y = predict(network, test[i])
    p = np.append(p, np.argmax(y))
    
#　正解データ    
t = np.argmax(test_labels[:NUM], axis=1)

# 予測結果と正解データの比較
print(p)
print(t)

# 識別精度
accuracy_score(t, p)

[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 6 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3 7 4 2 4 3 0 7 0 2 9
 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 4 3 1 4 1 7 6 9 6 0 5 4 9 9 2 1 9 4 8
 7 3 9 7 4 4 4 9 2 5 4 7 6 4 9 0 5 8 5 6 6 5 7 8 1 0 1 6 4 6 7 3 1 7 1 8 2
 0 9 9 9 5 5 1 5 6 0 3 4 4 6 5 4 6 5 4 5 1 4 4 7 2 3 2 7 1 8 1 8 1 8 5 0 8
 9 2 5 0 1 1 1 0 9 0 3 1 6 4 2 3 6 1 1 1 3 9 5 2 9 4 5 9 3 9 0 3 5 5 5 7 2
 2 7 1 2 8 4 1 7 3 3 8 7 7 9 2 2 4 1 5 8 8 7 2 5 0 2 4 2 4 1 9 5 7 7 2 8 2
 0 8 5 7 7 9 1 8 1 8 0 3 0 1 9 9 4 1 8 2 1 2 9 7 5 9 2 6 4 1 5 4 2 9 2 0 4
 0 0 2 8 6 7 1 2 4 0 2 7 4 3 3 0 0 5 1 9 6 5 2 5 7 7 9 3 0 4 2 0 7 1 1 2 1
 5 3 3 9 7 8 6 3 4 1 3 8 1 0 5 1 3 1 5 0 6 1 8 5 1 9 9 4 6 7 2 5 0 6 5 6 3
 7 2 0 8 8 5 4 1 1 4 0 7 3 7 6 1 6 2 1 9 2 8 6 1 9 5 2 5 4 4 2 8 3 8 2 4 5
 0 3 1 7 7 5 7 9 7 1 9 2 1 4 2 9 2 0 4 9 1 4 8 1 8 4 5 9 8 8 3 7 6 0 0 3 0
 2 0 6 4 8 5 3 3 2 3 9 1 2 6 8 0 5 6 6 6 3 8 8 2 7 5 8 9 6 1 8 4 1 2 8 3 1
 9 7 5 4 0 8 9 9 1 0 5 2 

0.932